In [1]:
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
import tensorflow_datasets as tfds
import warnings

Using TensorFlow backend.


In [3]:
# keras mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(test_images.shape)

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

network.fit(train_images, train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc: ', test_acc)

(60000, 28, 28)
(10000, 28, 28)
Epoch 1/5
60000/60000 [==============================] - 19s 322us/step - loss: 0.2558 - accuracy: 0.9267
Epoch 2/5
60000/60000 [==============================] - 17s 286us/step - loss: 0.1019 - accuracy: 0.9696
Epoch 3/5
60000/60000 [==============================] - 17s 277us/step - loss: 0.0659 - accuracy: 0.9808
Epoch 4/5
60000/60000 [==============================] - 16s 273us/step - loss: 0.0470 - accuracy: 0.9857
Epoch 5/5
10000/10000 [==============================] - 4s 377us/step
test_acc:  0.9782000184059143


In [7]:
mnist_builder = tfds.builder("mnist")

mnist_builder.download_and_prepare()

ds_train, ds_test = mnist_builder.as_dataset(split=[tfds.Split.TRAIN, tfds.Split.TEST])

In [8]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

n_train = ds_train
#n_validation = ds_validation
n_test = ds_test

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

weights = {
    "w1": tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    "w2": tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    "w3": tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    "out": tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

biases = {
    "b1": tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    "b2": tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    "b3": tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    "out": tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.add(tf.matmul(X, weights["w1"]), biases["b1"])
layer_2 = tf.add(tf.matmul(layer_1, weights["w2"]), biases["b2"])
layer_3 = tf.add(tf.matmul(layer_2, weights["w3"]), biases["b3"])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights["out"]) + biases["out"]

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
    )
)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(n_iterations):
    batch_x, batch_y = n_train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X:batch_x, Y: batch_y, keep_prob: dropout
    })
    
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
        )
        print("Iteration", str(i), "\t| loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

AttributeError: module 'tensorflow' has no attribute 'placeholder'